## Setup

In [1]:
import pprint
import networkx as nx
import random
import time
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np

from planners.IPBasicPRM import BasicPRM
from optimize_path import OptimizeFlyby
from planners.IPPerfMonitor import IPPerfMonitor
from collision_checker import CollisionChecker
from benchmarks import construct_benchmark_environments
from analysis import (
    create_interactive_viewer,
    interactive_radius_exploration,
    interactive_k_exploration,
    find_optimal_global_k_interactive,
    interactive_individual_optimization,
    render_initial_path,
    retrieve_path_positions,
    calculate_path_length,
    plot_performance_data,
    clear_graph_attributes
)

## Load Environments

In [2]:
env_dict = construct_benchmark_environments()

## Create Planner and Collision Checker for each environment

In [3]:
for name, item in env_dict.items():
    # 1. Initialize a CollisionChecker with the environment
    cc = CollisionChecker(item["env"])

    # 2. Initialize a Planner with the collisionChecker
    planner = BasicPRM(cc)

    # 3. Construct the NetworkX Graph from the predefined solution path
    G = nx.Graph()

    node_names = []

    for i, coord in enumerate(item["smooth_path"]):
        # Determine the name based on position in the list
        if i == 0:
            n = "start"
        elif i == len(item["smooth_path"]) - 1:
            n = "goal"
        else:
            n = f"{i}" # Naming middle nodes generically
        
        node_names.append(n)
        
        # Add node with the 'pos' attribute
        G.add_node(n, pos=coord)

    # Add edges connecting each node to the next
    for i in range(len(node_names) - 1):
        current_node = node_names[i]
        next_node = node_names[i+1]
        G.add_edge(current_node, next_node)

    # 4. Inject the constructed graph into the Planner instance
    planner.graph = G
    
    # 5. Add the node names and the planner to the environment dictionary
    env_dict[name]["solution_node_names"] = node_names
    env_dict[name]["planner"] = planner
    del env_dict[name]["smooth_path"]

# pprint.pprint(env_dict)

## Show the Environments

In [4]:
# 1. Define the worker function
def render_env(Env):
    item = env_dict[Env]
    planner = item["planner"]
    
    ax = planner._collisionChecker.draw_enviroments()
    path_pos = retrieve_path_positions(planner.graph, item['solution_node_names'])
    planner._collisionChecker.draw_path(path_pos, ax=ax)
    plt.title(f"Environment {Env}")
    plt.show()

# 2. Call the generic viewer with 1 parameter
config = {'Env': list(env_dict.keys())}
create_interactive_viewer(config, render_env)

## Interactive Radius Exploration (k = symmetric)

In [5]:
optimizer = OptimizeFlyby()

interactive_radius_exploration(env_dict, optimizer)

## Interactive k (Skew) Exploration (r = fixed)

In [6]:
optimizer = OptimizeFlyby()

interactive_k_exploration(env_dict, optimizer, r_fixed=0.5)

## Find optimal global k

In [7]:
optimizer = OptimizeFlyby()

find_optimal_global_k_interactive(env_dict, optimizer)

Dropdown(description='Select Env:', options=('1', '2', '3', '4'), value='1')

## Optimize individual k's

In [8]:
optimizer = OptimizeFlyby()

interactive_individual_optimization(env_dict, optimizer)

Starting Individual Corner Optimization...
--- Pass 1 ---
--- Pass 2 ---
Converged early.
Optimization Complete.
Starting Individual Corner Optimization...
--- Pass 1 ---
--- Pass 2 ---
Optimization Complete.
Starting Individual Corner Optimization...
--- Pass 1 ---
--- Pass 2 ---
Optimization Complete.
Starting Individual Corner Optimization...
--- Pass 1 ---
--- Pass 2 ---
Optimization Complete.


Dropdown(description='Select Env:', options=('1', '2', '3', '4'), value='1')